# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [3]:
import findspark
findspark.init('/home/julia/spark-3.3.0-bin-hadoop3')

In [38]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [5]:
import numpy
from pyspark.ml.regression import LinearRegression

In [37]:
training = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [7]:
training.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
training.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
training.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [10]:
training.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_idx')

In [14]:
training_indexed = indexer.fit(training).transform(training)

In [15]:
training_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_idx|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [55]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage','passengers','length',
 'cabins','passenger_density','Cruise_line_idx'], outputCol='features')

In [56]:
output = assembler.transform(training_indexed)
output

DataFrame[Ship_name: string, Cruise_line: string, Age: int, Tonnage: double, passengers: double, length: double, cabins: double, passenger_density: double, crew: double, Cruise_line_idx: double, features: vector]

In [58]:
final_data = output.select('features', 'crew')

In [59]:
final_data.show()
final_data.head(1)[0]

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55)

In [43]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [44]:
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               123|
|   mean| 7.946910569105702|
| stddev|3.5539355073351038|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               35|
|   mean|7.257428571428569|
| stddev|3.313142583939394|
|    min|             0.59|
|    max|            12.38|
+-------+-----------------+



In [45]:
lr = LinearRegression(labelCol='crew')

In [46]:
lr_model = lr.fit(train_data)

22/09/01 11:44:25 WARN Instrumentation: [94ee7554] regParam is zero, which might cause numerical instability and overfitting.


In [47]:
test_results = lr_model.evaluate(test_data)

In [48]:
test_results.rootMeanSquaredError

0.8622894974449737

In [60]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               123|
|   mean| 7.946910569105702|
| stddev|3.5539355073351038|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [49]:
# How much variance is explained
test_results.r2

0.9302707382401523

In [50]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[6.0,93.0,23.94,9...|
|[7.0,116.0,31.0,9...|
|[9.0,59.058,17.0,...|
|[9.0,105.0,27.2,8...|
|[9.0,113.0,26.74,...|
|[10.0,46.0,7.0,6....|
|[10.0,86.0,21.14,...|
|[10.0,138.0,31.14...|
|[11.0,58.6,15.66,...|
|[11.0,91.0,20.32,...|
|[11.0,91.62700000...|
|[12.0,108.865,27....|
|[14.0,30.27699999...|
|[14.0,33.0,4.9,5....|
|[14.0,63.0,14.4,7...|
|[14.0,76.8,19.6,8...|
|[14.0,77.104,20.0...|
|[15.0,70.367,20.5...|
|[16.0,78.491,24.3...|
+--------------------+
only showing top 20 rows



In [51]:
predictions = lr_model.transform(unlabeled_data)

In [52]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.286530259994258|
|[6.0,93.0,23.94,9...|10.588337250578588|
|[7.0,116.0,31.0,9...| 12.70143023311377|
|[9.0,59.058,17.0,...| 7.607925817568592|
|[9.0,105.0,27.2,8...|11.283476916941254|
|[9.0,113.0,26.74,...|11.361575862538798|
|[10.0,46.0,7.0,6....|2.7002784760014222|
|[10.0,86.0,21.14,...| 9.678321935349286|
|[10.0,138.0,31.14...|13.009702745335312|
|[11.0,58.6,15.66,...| 7.421996404246935|
|[11.0,91.0,20.32,...| 9.173930378377548|
|[11.0,91.62700000...| 9.179609912216847|
|[12.0,108.865,27....|10.841667625210608|
|[14.0,30.27699999...| 3.428101199381499|
|[14.0,33.0,4.9,5....| 2.910821447470879|
|[14.0,63.0,14.4,7...|    6.656354799171|
|[14.0,76.8,19.6,8...| 8.851506115613969|
|[14.0,77.104,20.0...| 8.761732043814943|
|[15.0,70.367,20.5...| 8.670214576018175|
|[16.0,78.491,24.3...| 8.182951520519607|
+--------------------+------------

In [61]:
from pyspark.sql.functions import corr

In [63]:
training.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [64]:
# Where one is biggest correlations
training.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [65]:
training.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

